In [1]:
# import the libraries
import os
import scipy.io
import scipy.misc
#various utilities that dont have another home
import numpy as np
import pandas as pd
import PIL
import struct


In [2]:
import cv2
from numpy import expand_dims
#expand the shape of an array


In [4]:
import tensorflow as tf
from skimage.transform import resize
from keras import backend as k
from keras.layers import Input , Lambda , Conv2D , BatchNormalization,LeakyReLU, ZeroPadding2D , UpSampling2D


In [6]:
from keras.models import load_model , Model
from keras.layers import add, concatenate
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib.patches import Rectangle
%matplotlib inline


In [1]:
class WeightReader:
    def __init__(self,weight_file):
        with open(weight_file, 'rb') as w_f:
            major, = struct.unpack('i', w_f.read(4))
            minor, = struct.unpack('i', w_f.read(4))
            revision, =struct.unpack('i',w_f.read(4))

            if (major*10 + minor)>=2 and major <1000 and minor <1000:
                w_f.read(8)
            else:
                w_f.read(4)

            transpose = (major >1000 ) or (minor > 1000)

            binary = w_f.read()
        self.offset = 0
        self.all_weights = np.frombuffer(binary , dtype= 'float32')

    def read_bytes(self, size):
        self.offset = self.offset + size
        return self.all_weights[self.offset-size : self.offset]
    
    def load_weights(self, model):
        for i in range(106):
            try:
                conv_layer = model.get_layer('conv_'+str(i))
                print("loading weights of convolution #"+str(i))

                if i not in [81,93,105]:
                    norm_layer = model.get_layer('bnorm_'+ str(i))

                    size = np.prod(norm_layer.get_weights()[0].shape)

                    beta = self.read_bytes(size) #bias
                    gamma  = self.read_bytes(size)# scale
                    mean = self.read_bytes(size) #mean
                    var= self.read_bytes(size) #variance

                    weights = norm_layer.set_weights([gamma, beta , mean , var])
                
                if len(conv_layer.get_weights())>1:
                    bias = self.read_bytes(np.prod(conv_layer.get_weights()[1].shape))
                    kernel = self.read_bytes(np.prod(conv_layer.weights()[0].shape))

                    kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
                    kernel = kernel.transpose([2,3,1,0])
                    conv_layer.set_weights([kernel , bias])
                
                else:
                    kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
                    kernel = self.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
                    conv_layer.set_weights([kernel])
            except ValueError:
                print("no convolution #"+str(i) )
    def reset(self):
        self.offset = 0  

In [2]:
def conv_block(inp , convs , skip = True):
    x= inp
    count=0

    for conv in convs:
        if count == (len(convs)-2) and skip:
            skip_connection = x
        count+=1

        if conv['stride']>1 : x = ZeroPadding2D(((1,0),(1,0)))(x)
        #peculiar padding added to darknet as it prefer left and top
        x = Conv2D(conv['filter'],
                   conv['kernel'],
                   stride = conv['stride'],
                   padding = 'valid ' if conv['stride']>1 else'same',
                   name = 'conv_' + str(conv['layer_idx']),
                   use_bias= False if conv['bnorm'] else True)(x)
        
        if conv['bnorm']: x = BatchNormalization(epsilon= 0.001, name = 'bnorm_' + str(conv['layer_idx']))(x)
        if conv['leaky']:x = LeakyReLU(alpha = 0.1 , name = 'leaky_'+ str(conv['layer_idx']))(x)
    return add([skip_connection, x]) if skip else x
#peculiar padding as darknet prefer left and top


In [5]:
#creatinng the yolo model
def make_yolov3_model():
    input_image = Input(shape = (None , None ,3))

    #Layer 0 =>4
    x = _conv_block(input_image,[{'filter': 32 , 'kernel': 3 , 'stride': 1, 'bnorm' :True , 'leaky': True , 'layer_idx':0},
                                  {'filter': 64 , 'kernel': 3 , 'stride' :2 , 'bnorm':True , 'leaky':True , 'layer_idx':1},
                                  {'filter': 32 , 'kernel':1 , 'stride' :1, 'bnorm':True , 'leaky':True , 'layer_idx':2},
                                  {'filter': 64 , 'kernel':3 , 'stride':1 , 'bnorm':True , 'leaky': True , 'layer_idx':3}])
    #layer 5=>8
    x = _conv_block(x , [{'filter':128 , 'kernel':3  , 'stride':2 , 'bnorm':True , 'leaky': True , 'layer_idx':5},
                         {'filter': 64, 'kernel':1 , 'stride':1, 'bnorm': True , 'leaky':True, 'layer_idx':6},
                         {'filter':128 , 'kernel':3 , 'stride':1 , 'bnorm':True, 'leaky':True , 'layer_idx':7 }])
    
    #layer 9=>11
    x = _conv_block(x, [{'filter': 64 , 'kernel':1 , 'stride':1 , 'bnorm':True ,'leaky':True , 'layer_idx':9},
                        {'filter': 128 , 'kernel':3 , 'stride':1 , 'bnorm':True, 'leaky':True , 'layer_idx':10}])
    
    # layer 12 =>15
    x = _conv_block(x ,[{'filter':256 , 'kernel':3  , 'stride':2 , 'bnorm':True , 'leaky': True , 'layer_idx':12},
                         {'filter': 128, 'kernel':1 , 'stride':1, 'bnorm': True , 'leaky':True, 'layer_idx':13},
                         {'filter':256 , 'kernel':3 , 'stride':1 , 'bnorm':True, 'leaky':True , 'layer_idx':14}])
    
    #layer 16 =>46
    for i in range(7):
        x = _conv_block(x , [{'filter':128 , 'kernel':1  , 'stride':1 , 'bnorm':True , 'leaky': True , 'layer_idx':16 + i*3},
                         {'filter': 256, 'kernel':3 , 'stride':1, 'bnorm': True , 'leaky':True, 'layer_idx':17 + i*3}])
        
    skip_36 = x

    #layer 37=>40
    x = _conv_block(x, [{'filter':512 , 'kernel':3  , 'stride':2 , 'bnorm':True , 'leaky': True , 'layer_idx':37},
                         {'filter': 256, 'kernel':1 , 'stride':1, 'bnorm': True , 'leaky':True, 'layer_idx':38},
                         {'filter':512 , 'kernel':3 , 'stride':1 , 'bnorm':True, 'leaky':True , 'layer_idx':39}])
    
    # layer 41 => 61
    for i in range(7):
        x = _conv_block(x , [{'filter':256 , 'kernel':1  , 'stride':1 , 'bnorm':True , 'leaky': True , 'layer_idx':41 + i*3},
                         {'filter': 512, 'kernel':3 , 'stride':1, 'bnorm': True , 'leaky':True, 'layer_idx': 42 + i*3}])
        
    skip_61 = x 
    
    #layer 62 => 65
    x = _conv_block(x,[{'filter':1024 , 'kernel':3  , 'stride':2 , 'bnorm':True , 'leaky': True , 'layer_idx':62},
                       {'filter': 512, 'kernel':1 , 'stride':1, 'bnorm': True , 'leaky':True, 'layer_idx': 63},
                       {'filter':1024 , 'kernel':3 , 'stride':1 , 'bnorm':True, 'leaky':True , 'layer_idx':64 }])
    
    #layer 66 =>74

    for i in range(3):
        x = _conv_block(x , [{'filter':512 , 'kernel':1  , 'stride':1 , 'bnorm':True , 'leaky': True , 'layer_idx': 66+i*3},
                         {'filter': 1024, 'kernel':3 , 'stride':1, 'bnorm': True , 'leaky':True, 'layer_idx': 67+ i*3}])
        
    #layer 75 =>79
    x = _conv_block(x ,[{'filter':512 , 'kernel':1 , 'stride':1, 'bnorm':True , 'leaky': True , 'layer_idx':75},
                         {'filter': 1024, 'kernel':3 , 'stride':1, 'bnorm': True , 'leaky':True, 'layer_idx':76},
                         {'filter':512 , 'kernel':1 , 'stride':1 , 'bnorm':True, 'leaky':True , 'layer_idx':77},
                         {'filter':1024 , 'kernel':3  , 'stride':1 , 'bnorm':True , 'leaky': True , 'layer_idx':78},
                         {'filter': 512, 'kernel':1 , 'stride':1, 'bnorm': True , 'leaky':True, 'layer_idx':79}] , skip = False)
    
    #layer 80 =>82
    yolo_82 = _conv_block(x , [{'filter':1024 , 'kernel':3  , 'stride':1 , 'bnorm':True , 'leaky': True , 'layer_idx':80},
                               {'filter': 225, 'kernel':1 , 'stride':1, 'bnorm': False , 'leaky':False, 'layer_idx':81}])
    
    #layer 83 =>86
    x = _conv_block( x , [{'filter': 1024 , 'kernel': 3, 'stride' : 1, 'bnorm': True , 'leaky': True, 'layer_idx': 84}], skip = False)
    x = UpSampling2D(2)(x)
    x= concatenate([x, skip_61])

    #layer 87 = >91
    x = _conv_block(x , [{'filter':256 , 'kernel':1  , 'stride':1 , 'bnorm':True , 'leaky': True , 'layer_idx':87},
                         {'filter': 512, 'kernel':3 , 'stride':1, 'bnorm': True , 'leaky':True, 'layer_idx':88},
                         {'filter':256 , 'kernel':1 , 'stride':1 , 'bnorm':True, 'leaky':True , 'layer_idx':89},
                         {'filter':512 , 'kernel':3  , 'stride':1 , 'bnorm':True , 'leaky': True , 'layer_idx':90},
                         {'filter': 256, 'kernel':1 , 'stride':1, 'bnorm': True , 'leaky':True, 'layer_idx':91}], skip= False)
    
    #layer 92=>94
    yolo_94 = _conv_block(x ,[{'filter':512 , 'kernel':3  , 'stride':1 , 'bnorm':True , 'leaky': True , 'layer_idx':92},
                              {'filter': 255, 'kernel':1 , 'stride':1, 'bnorm': False , 'leaky':False, 'layer_idx':93}], skip = False)
    
    #layer 95=>98
    x = _conv_block( x, [{'filter':128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx':96}], skip = False)
    x = UpSampling2D(2)(x)
    x = concatenate([x, skip_36])

    #layer 99 =>106
    yolo_106 = _conv_block(x , [{'filter':128 , 'kernel':1  , 'stride':1, 'bnorm':True , 'leaky': True , 'layer_idx':99},
                                {'filter':256 , 'kernel':3 , 'stride':1, 'bnorm': True , 'leaky':True, 'layer_idx':100},
                                {'filter':128 , 'kernel':1 , 'stride':1 , 'bnorm':True, 'leaky':True , 'layer_idx':101 },
                                {'filter':256 , 'kernel':3  , 'stride':1 , 'bnorm':True , 'leaky': True , 'layer_idx': 102},
                                {'filter':128, 'kernel':1 , 'stride':1, 'bnorm': True , 'leaky':True, 'layer_idx':103},
                                {'filter':256 , 'kernel':3 , 'stride':1 , 'bnorm':True, 'leaky':True , 'layer_idx':104 },
                                {'filter':255 , 'kernel':1 , 'stride':1 , 'bnorm':False, 'leaky':False , 'layer_idx':105 }])
    model = Model(input_image , [yolo_82 , yolo_94 , yolo_106])
    return model
